<a href="https://colab.research.google.com/github/AromaR/685_Project/blob/main/Copy_of_Copy_of_POMO_Probing_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
import torch
assert torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")

Found device: Tesla P100-PCIE-16GB, n_gpu: 1


In [3]:
import pandas as pd
import numpy as np
!pip install transformers
!pip install tqdm
from tqdm import tqdm

     |████████████████████████████████| 1.4MB 9.2MB/s 
     |████████████████████████████████| 2.9MB 31.6MB/s 
     |████████████████████████████████| 890kB 44.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6d110508d84f5f1d09157c855f734a8fce9f8f9d41c220ee8b1c6e32eedc5693
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model = BertModel.from_pretrained("bert-base-uncased")


In [5]:
dataset = pd.read_csv("/content/drive/My Drive/POMO/train.pm",sep='\\t')
dataarr = []
ansarr = []
for i in range(5000,10000):
  dataarr.append(dataset.iloc[i,0])
  ansarr.append(0)
  dataarr.append(dataset.iloc[i,3])
  ansarr.append(1)
torch.set_printoptions(profile="full")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [ ]:
with open("/content/drive/My Drive/POMO/emb_05_train_base", "w") as writer:
  for i in tqdm(range(0, len(dataarr), 128)):
    batch = dataarr[i:i+128]
    ans = ansarr[i:i+128]
    inputs = tokenizer.batch_encode_plus(batch,
                add_special_tokens=True,
                truncation=True,
                padding=True,
                return_tensors="pt",
            )
    for j in range(0,128):
      try:
        writer.write("|")
        writer.write(str(model(torch.tensor(inputs['input_ids'][j]).unsqueeze(0))[0][0][0]))
        writer.write("$")
        writer.write(str(ans[j]))
      except:
        continue
torch.set_printoptions(profile="default")
    


  0%|          | 0/79 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
 44%|████▍     | 35/79 [34:03<40:24, 55.10s/it]

In [ ]:
emb_data = tokenizer.batch_encode_plus(dataarr[0:10])

In [ ]:
print(len(emb_data['input_ids']))

In [ ]:
h = torch.tensor(tokenizer.encode(dataset.iloc[i,0])).unsqueeze(0)
print(h)

In [ ]:
print(len(emb_data['input_ids'][0]))

In [ ]:
mods = []
for i in range(0,10):
  mods.append(model(torch.tensor(emb_data['input_ids'][i]).unsqueeze(0))[0][0][0])
  torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
mods = torch.stack((mods))
torch.cuda.empty_cache()

In [ ]:
print(len(mods[1]))

In [ ]:
#with open('/content/drive/My Drive/POMO/train.pm',"r") as f:
#   print(f.read())


In [ ]:
dataset = pd.read_csv("/content/drive/My Drive/POMO/test.pm",sep='\\t')
dataarr_test = []
ansarr_test = []
print("here we add to a 2d array, a sentence and a number 1/0 according to whether it contains appositions. The first sentence at pos 0 is not appositional, the one at position 3 is")
for i in range(0,len(dataset)):
  dataarr_test.append(dataset.iloc[i,0])
  ansarr_test.append(0)
  dataarr_test.append(dataset.iloc[i,3])
  ansarr_test.append(1)

In [ ]:
dataset = pd.read_csv("/content/drive/My Drive/POMO/valid.pm",sep='\\t')
dataarr_valid = []
ansarr_valid = []
print("here we add to a 2d array, a sentence and a number 1/0 according to whether it contains appositions. The first sentence at pos 0 is not appositional, the one at position 3 is")
for i in range(0,len(dataset)):
  dataarr_valid.append(dataset.iloc[i,0])
  ansarr_valid.append(0)
  dataarr_valid.append(dataset.iloc[i,3])
  ansarr_valid.append(1)

In [ ]:
print(len(dataarr))
print(dataarr[0])
print(ansarr[0])
print(dataarr[1])
print(ansarr[1])
h = []
for j in ansarr:
  h.append(torch.LongTensor([j]).unsqueeze(0))
ansarr1 = torch.stack((h))
print(ansarr1.shape)

In [ ]:
print(len(dataarr_test))
print(dataarr_test[0])
print(ansarr_test[0])
print(dataarr_test[1])
print(ansarr_test[1])
ansarr1_test = torch.from_numpy(np.array(ansarr_test))

In [ ]:
print(len(dataarr_valid))
print(dataarr_valid[0])
print(ansarr_valid[0])
print(dataarr_valid[1])
print(ansarr_valid[1])
ansarr1_valid = torch.from_numpy(np.array(ansarr_valid))

Creation of data

Conversion to hugging faces BERT-Tacred fine-tuned representation

In [ ]:
class POMO_Dataset(torch.utils.data.dataset.Dataset):
    def __init__(self, _dataset):
        self.dataset = _dataset

    def __getitem__(self, index):
        example= self.dataset[0][index]
        target = self.dataset[1][index]
        target.squeeze(0)
        return np.array(example), target

    def __len__(self):
        return len(self.dataset)

In [ ]:
max_len = 1024

In [ ]:
dataarr_test[2]

In [ ]:
test = []
data_test = []
for i in dataarr_test:
  try:
    input_ids = torch.tensor(tokenizer.encode(i)).unsqueeze(0)  # Batch size 1
    outputs = model(input_ids)
    data_test.append(outputs[0][0])
  except:
    print(i)
    break
test.extend([torch.tensor(data_test).unsqueeze(0).unsqueeze(0),ansarr1_test])


In [ ]:
emb_data = tokenizer.batch_encode_plus(dataarr_valid)
k = [torch.LongTensor(x) for x in emb_data['input_ids']]
max_len = max([len(x) for x in emb_data['input_ids']])
data = [torch.nn.functional.pad(x, pad=(0, max_len - x.numel()), mode='constant', value=0) for x in k]
data = torch.stack(data)
valid = []
for i in range(0,len(data)):
  valid.append([data[i],ansarr1_valid[i]])


In [ ]:
emb_data = tokenizer.batch_encode_plus(dataarr)
k = [torch.LongTensor(x) for x in emb_data['input_ids']]
max_len = max([len(x) for x in emb_data['input_ids']])
data = [torch.nn.functional.pad(x, pad=(0, max_len - x.numel()), mode='constant', value=0) for x in k]
data = torch.stack(data)
train = []



Get Bert embeddings

In [ ]:
input_ids = torch.tensor(tokenizer.encode("Hello, my name is Aroma")).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]

In [ ]:
print(len(last_hidden_states[0][0])) #CLS baby!

In [ ]:
ansarr.unsqueeze(0).unsqueeze(0)

In [ ]:
a = []
for i in data:
  a.append(i.unsqueeze(0).unsqueeze(0))

In [ ]:
a = torch.stack(a)

In [ ]:
print(a.shape)

In [ ]:
train = []
train = [a,ansarr1]

In [ ]:
print(data)

In [ ]:
print((train[0][1]))

In [ ]:

test_loader = torch.utils.data.DataLoader(dataset=POMO_Dataset(test),
                                           batch_size=4,
                                           shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=POMO_Dataset(valid),
                                           batch_size=4,
                                           shuffle=True)
train_loader = torch.utils.data.DataLoader(dataset=POMO_Dataset(train),
                                           batch_size=4,
                                           shuffle=True)


  
  


In [ ]:
for batch_images, targets in train_loader:
    print(batch_images.shape)
    print(targets)
    break

Probing task level 1 : Classifier - binary, Aroma's CNN

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 1,)
        self.fc1 = nn.Linear(801, 120)
        self.fc2 = nn.Linear(120, 1)

    def forward(self, x):
        x = (F.relu(self.conv1(x.long()))).long()
        x = F.relu(self.fc1(x)).long()
        x = (self.fc2(x)).long()
        return x


net = Net()
    
   

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.long())
        print(labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
!pip install keras

In [ ]:
from keras.models import Sequential

from keras.layers import Dense, Conv2D, Flatten

model = Sequential()

model.add(Conv2D(64, kernel_size=1, activation='relu', input_shape=(1,1,801)))

model.add(Conv2D(32, kernel_size=1, activation='relu'))

model.add(Flatten())

model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train, ansarr, validation_data=(valid, ansarr_valid), epochs=3)

In [ ]:
print(data[0])

In [ ]:
print(data[0].numel())